# Select

Selecting data is the most important aspect of SQL for database users. This page provides an overview of different statements that can be used in the `SELECT` expression.

We'll use PostgreSQL as the standard; other databases will be considered in specific sections.

In [12]:
docker run -d --rm\
    --name select_examples\
    -e POSTGRES_PASSWORD=postgres\
    postgres:15.4 &> /dev/null
sleep 5

**Note**: Don't forget to stop the container when finished.

In [10]:
docker stop select_examples

select_examples


## IS (NOT) NULL

You can check if value (not) taking NULL value using following suntax `<value> IS (NOT) NULL`.

---

The following example shows the original column and the application of `IS NULL` and `IS NOT NULL` to it.

In [15]:
docker exec -i select_examples psql -U postgres -d postgres << EOF
SELECT
    numeric_value,
    numeric_value IS NULL is_null,
    numeric_value IS NOT NULL is_not_null
FROM (
    VALUES (NULL), (1), (2), (NULL), (4), (5)
) AS temp(numeric_value);
EOF

 numeric_value | is_null | is_not_null 
---------------+---------+-------------
               | t       | f
             1 | f       | t
             2 | f       | t
               | t       | f
             4 | f       | t
             5 | f       | t
(6 rows)



## Conditional (CASE)

With the `CASE` keyword, you can assign specified values to logical conditions. Find out more in the [specific page](select/conditional.ipynb).

---

The following cell shows how you can assign a string that describes the value originally in the column.

In [9]:
docker exec -i select_examples psql -U postgres -d postgres << EOF
SELECT
    numeric_value,
    CASE
        WHEN numeric_value<2 THEN 'less than two'
        ELSE 'more or equals than two'
    END
FROM (
    VALUES (0), (1), (2), (3), (4), (5)
) AS temp(numeric_value);
EOF

 numeric_value |          case           
---------------+-------------------------
             0 | less than two
             1 | less than two
             2 | more or equals than two
             3 | more or equals than two
             4 | more or equals than two
             5 | more or equals than two
(6 rows)

